In [ ]:
!pip3 install python-sat

In [ ]:
import numpy as np
import time
from random import randrange
from random import seed
from random import shuffle
from pysat.solvers import Glucose4
from pysat.solvers import Minisat22
from pysat.solvers import MapleCM
from pysat.solvers import Lingeling
from pysat.solvers import Minicard
import matplotlib.pyplot as plt
import math
import networkx as nx
import random
import sympy
from sympy.logic import POSform
from sympy import symbols
w, x, y, z, a = symbols('w x y z a')

## Implementation

In [ ]:
def print_grid(grid, m):
    count = 0
    for elem in grid:
        print(elem, end=" ")
        count = (count + 1)%m
        if count==0:
            print()
    print()

class Graph:

    def __init__(self, n):
        self.n = n
        self.adj = [[] for i in range(n)]
        self.degree = 0    #maximum degree
        self.negative = set()

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)
        if len(self.adj[u])>self.degree or len(self.adj[v])>self.degree:
            self.degree+=1

    #all zero state
    def find_coloring(self):
        colors = [0 for _ in range(self.n)]
        return colors


    '''
        Forward simulation of the heat-bath dynamics
    '''

    def heatbath_dynamics(self, coloring, transitions):

        for (node, interval) in reversed(transitions):

            count = 0
            for u in self.adj[node]:
                negate = 1 if (node, u) in graph.negative else 0
                count += (coloring[u] + negate) % 2

            if count >= interval:
                coloring[node] = 1
            else:
                coloring[node] = 0

        return coloring



    # Return a set 'fam' of vertices with distance at most R from v
    # and a set 'brdr' of all vertices in 'fam' with neighbors outside of 'fam'.

    def family(self, v, R):

        visited = {v}   # Set of visited nodes.
        queue = [v]     # Initialize a queue

        for _ in range(R):
            new_queue = []
            for u in queue:
                for neigh in self.adj[u]:
                    if neigh not in visited:
                        visited.add(neigh)
                        new_queue.append(neigh)
            queue = new_queue.copy()

        border = set()
        for u in queue:
            for neigh in self.adj[u]:
                if neigh not in visited:
                    border.add(u)
                    break

        return visited, border

In [ ]:
#clauses to ensure that: the number of neighbours of v with spin 1
# is larger than 'interval' <=> the spin of v is set to 1
def interval_to_clauses(interval):

    #define minterms of clause
    minterms = []
    for X in [0,1]:
      for Y in [0,1]:
        for Z in [0,1]:
          for W in [0,1]:
            if X+Y+Z+W>=interval:
                A = 1
            else:
                A = 0
            minterms.append([X,Y,Z,W,A])

    #print(minterms)
    expression = str(POSform([w, x, y, z, a], minterms))
    #print(expression)

    # Define the variable map
    variable_map = {'w': 1, 'x': 2, 'y': 3, 'z': 4, 'a': 5}
    # Remove spaces for easier processing
    expression = expression.replace(" ", "")
    # Split by '&' to get individual clauses
    clauses = expression.split('&')
    result = []

    for clause in clauses:
        # Remove outer parentheses
        clause = clause.strip('()')
        # Split by '|'
        literals = clause.split('|')
        clause_list = []

        for literal in literals:
            # Handle negation
            if literal.startswith('~'):
                clause_list.append(-variable_map[literal[1]])  # Negated variable
            else:
                clause_list.append(variable_map[literal])  # Positive variable
        result.append(clause_list)

    return result

learnt_intervals = [interval_to_clauses(i) for i in range(0,6)]

#print(interval_to_clauses(2))

#checks whether a collapse has occured
def CFTPcollapse_ising(graph, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i):
        return i + 1
    #color after transition k
    def P(k):
        return n + k + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring()
    #print("primary coloring:", init_state)
    final_state = graph.heatbath_dynamics(init_state.copy(), transitions)
    #print_grid(final_state,m)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()


    for k in range(t):

        node, interval = transitions[-k-1] # apply the transitions in reverse order

        clauses = learnt_intervals[interval]

        variable_map = {}
        # If S(u)=1 for some neighbour u, P(k) is set to 0
        curr = 1

        for u in graph.adj[node]:
            negate = -1 if (node, u) in graph.negative else 1
            if (d[u] == -1):
                variable_map[curr]=S(u)*negate
                variable_map[-curr]=-S(u)*negate
            else:
                variable_map[curr]=P(d[u])*negate
                variable_map[-curr]=-P(d[u])*negate
            curr+=1

        variable_map[5]=P(k)
        variable_map[-5]=-P(k)

        #print(variable_map)
        #print(clauses)
        new_clauses = [[variable_map[literal] for literal in clause ] for clause in clauses ]
        #print(new_clauses)

        for clause in new_clauses:
            g.add_clause(clause)

        d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(n):
        if (d[i] == -1):
            clause.append(-(final_state[i]*2-1)*S(i))
        else:
            clause.append(-(final_state[i]*2-1)*P(d[i]))

    g.add_clause(clause)

    time_solver = time.time()
    # Solve the CSP
    status = g.solve()
    print("SAT solver time:", time.time() - time_solver)

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]
        #print(model)
        for u in range(n):
            if model[u]>0:
                coloring[u] = 1
            else:
                coloring[u] = 0
        #print(coloring)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


def find_interval(u, lamda):
    a, b, c, d, e = 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1)

    #(start, end, label)
    intervals = [
        (0, a, 0),
        (a, b, 1),
        (b, c, 2),
        (c, d, 3),
        (d, e, 4),
        (e, 1, 5),
    ]

    for start, end, label in intervals:
        if start <= u < end:
            return label

    print("ERROR")
    return


#lamda>1
def CFTPsolver_ising(graph, lamda, solver_name):
    n = graph.n
    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = randrange(n)
            u = random.uniform(0, 1)
            interval = find_interval(u,lamda)
            transitions.append((node, interval))

        time_before = time.time()

        print(t)

        ret_value = CFTPcollapse_ising(graph, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.heatbath_dynamics(coloring1, transitions)
            coloring2 = graph.heatbath_dynamics(coloring2, transitions)
            if coloring1==coloring2:
                print("ERROR: coloring1==coloring2")
                print_grid(coloring1,m)

        print("time:", time.time() - time_before)
        t*=2

    return transitions

## Experiments

In [ ]:
#Construct a periodic m x m grid

m = 30
n = m*m

graph = Graph(n)
seed(10)
prob = 0   #the probability of making an edge interaction antiferromagnetic

def add_negative(x,y):
    u = random.uniform(0, 1)
    if u < prob:
        graph.negative.add((x, y))
        graph.negative.add((y, x))

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)
        add_negative(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)
        add_negative(m*i + j, m*i+m + j)

for j in range(m):
    graph.add_edge(m*j, m*j+m-1)
    add_negative(m*j, m*j+m-1)

for i in range(m):
    graph.add_edge(i, m*(m-1) + i)
    add_negative(i, m*(m-1) + i)

print(graph.negative)
print(len(graph.negative)//2)

set()
0


In [ ]:
lamda = 2.3   # λ=e^2β. phase transition for λ_c=2.414

seed(21)

solvers = ["glucose"]

lst_x_axis = []
lst_stamps = []

print("intervals:", 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1))

for solver_name in solvers:

    transitions = CFTPsolver_ising(graph, lamda, solver_name)

    coloring1 = graph.find_coloring()
    coloring2 = coloring1.copy()
    for i in range(n):
        #coloring2[i] = '?'
       coloring2[i] = 1 - coloring1[i]
    coloring1 = graph.heatbath_dynamics(coloring1, transitions)
    coloring2 = graph.heatbath_dynamics(coloring2, transitions)
    #print(transitions)
    print(coloring1 == coloring2, "should be True")
    print_grid(coloring1,m)
    print_grid(coloring2,m)

intervals: 0.034501675056324 0.15898251192368842 0.5 0.8410174880763116 0.965498324943676
1
SAT solver time: 0.00014472007751464844
FEASIBLE
time: 0.0013637542724609375
2
SAT solver time: 0.00011968612670898438
FEASIBLE
time: 0.0012080669403076172
4
SAT solver time: 7.748603820800781e-05
FEASIBLE
time: 0.0012497901916503906
8
SAT solver time: 9.632110595703125e-05
FEASIBLE
time: 0.0012290477752685547
16
SAT solver time: 0.00010037422180175781
FEASIBLE
time: 0.004805564880371094
32
SAT solver time: 9.34600830078125e-05
FEASIBLE
time: 0.001367330551147461
64
SAT solver time: 0.00011444091796875
FEASIBLE
time: 0.005868196487426758
128
SAT solver time: 0.00020551681518554688
FEASIBLE
time: 0.006901264190673828
256
SAT solver time: 0.00039505958557128906
FEASIBLE
time: 0.010790824890136719
512
SAT solver time: 0.00035858154296875
FEASIBLE
time: 0.013204336166381836
1024
SAT solver time: 0.0006093978881835938
FEASIBLE
time: 0.017294645309448242
2048
SAT solver time: 0.0011396408081054688
FEA

In [ ]:
#CFTP using monotonicity (only for ferromagnetic for general graphs)

seed(21)

lamda=2.3
print("intervals:", 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1))

ret_value = 1
transitions = []
t = 1
while(ret_value is not None):
    print(t)
    while(len(transitions)<t):
        # Select a random "Glauber" transition
        node = randrange(n)
        u = random.uniform(0, 1)
        interval = find_interval(u,lamda)
        transitions.append((node, interval))

    time_before = time.time()
    coloring1 = [0 for _ in range(n)]
    coloring2 = [1 for _ in range(n)]
    coloring1out = graph.heatbath_dynamics(coloring1, transitions)
    coloring2out = graph.heatbath_dynamics(coloring2, transitions)
    print("time:", time.time() - time_before)

    if(coloring1out == coloring2out):
        ret_value = None
    t*=2

print(t)
print_grid(coloring1out, m)

intervals: 0.034501675056324 0.15898251192368842 0.5 0.8410174880763116 0.965498324943676
1
time: 9.584426879882812e-05
2
time: 0.00014519691467285156
4
time: 9.584426879882812e-05
8
time: 0.00010561943054199219
16
time: 0.00012540817260742188
32
time: 0.0001659393310546875
64
time: 0.00035190582275390625
128
time: 0.0004734992980957031
256
time: 0.0007808208465576172
512
time: 0.0014426708221435547
1024
time: 0.0028557777404785156
2048
time: 0.01093602180480957
4096
time: 0.01889777183532715
8192
time: 0.044649362564086914
16384
time: 0.08325386047363281
32768
time: 0.17038416862487793
65536
time: 0.2643907070159912
131072
time: 0.6044819355010986
262144
time: 1.0590293407440186
524288
time: 1.489816665649414
1048576
time: 1.2861661911010742
2097152
0 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 1 0 0 0 
0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 
0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 